In [1]:
from sqlalchemy import create_engine
import os

SERVER:str = os.getenv("IP")
USERNAME: str = os.getenv("USERNAME")
PASSWORD: str = os.getenv("PASSWORD")
DATABASE: str = os.getenv("DATABASE_NAME")
ENCODED_PASSWORD: str = os.getenv("ENCODED_PASSWORD")

connection_uri = f'mssql+pymssql://{USERNAME}:{ENCODED_PASSWORD}@{SERVER}/{DATABASE}'
engine = create_engine(connection_uri, pool_size=5, max_overflow=10)


## Initialising Ollama llm

In [1]:
from llama_index.llms.ollama import Ollama
ollama_llm = Ollama(model="llama3.2:latest", request_timeout=100)

## initializing google gemini api model

In [56]:
from llama_index.llms.gemini import Gemini
import os

gemini_llm = Gemini(
    model="models/gemini-pro",
    api_key=os.getenv("GOOGLE_API_KEY"),
)


Backpack of wonders, secrets untold,
Holds dreams and magic, stories unfold.
Its depths a portal, to realms unknown,
Where imagination's seeds are sown.


## LLAMA_INDEX SQL

In [25]:
from llama_index.core import Settings
# from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


# Settings.node_parser = SentenceSplitter(chunk_size=512)

In [64]:
Settings.llm = ollama_llm

In [58]:
Settings.llm = gemini_llm


In [65]:
Settings.llm

Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7a4be20cd4b0>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x7a4be83ad750>, completion_to_prompt=<function default_completion_to_prompt at 0x7a4be47a2050>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='llama3.2:latest', temperature=0.75, context_window=3900, request_timeout=100.0, prompt_key='prompt', json_mode=False, additional_kwargs={}, is_function_calling_model=True, keep_alive=None)

In [26]:
Settings.embed_model = HuggingFaceEmbedding(model_name= "sentence-transformers/all-mpnet-base-v2")

In [27]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine, include_tables=["Customer", "LMS_Loan_Master"])

In [28]:
# from llama_index.llms.ollama import Ollama
# from llama_index.core.query_engine import NLSQLTableQueryEngine
# from sqlalchemy import create_engine
# import logging

# def setup_nl_sql_engine(engine, model_name: str = "llama2:latest"):
#     try:
#         # Initialize Ollama LLM
#         llm = Ollama(
#             model=model_name,
#             request_timeout=100
#         )
        
#         # Setup database connection
        
#         # Initialize SQL database with specific tables
#         sql_database = SQLDatabase(
#             engine,
#             include_tables=["Customer", "LMS_Loan_Master"]
#         )
        
#         # Create query engine
#         query_engine = NLSQLTableQueryEngine(
#             sql_database=sql_database,
#             tables=["Customer", "LMS_Loan_Master"],
#             llm=llm
#         )
        
#         return query_engine
        
#     except Exception as e:
#         logging.error(f"Error setting up NL-SQL engine: {str(e)}")
#         raise

# def query_database(query_engine, question: str) -> str:
#     try:
#         response = query_engine.query(question)
#         return str(response)
#     except Exception as e:
#         logging.error(f"Error executing query: {str(e)}")
#         return f"Error processing query: {str(e)}"

# # Example usage
# if __name__ == "__main__":
#     engine = setup_nl_sql_engine(engine)
    
#     questions = [
#         "which customer have loans more than 500000?",
#     ]
    
#     for question in questions:
#         print(f"Q: {question}")
#         print(f"A: {query_database(engine, question)}\n")

In [97]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["Customer", "LMS_Loan_Master"], llm=gemini_llm
)
# query_str = "show me all customers whose loan amount is greater than 500000"
query_str = "lap loans between 2019 to 2020"
response = query_engine.query(query_str)


In [98]:
print(response.metadata["sql_query"])

SELECT
  Loan_Id,
  Application_No,
  Application_LoanAmount,
  Application_LoanTenure,
  Application_LoanEMIAmount,
  Application_LoanIRRate,
  Application_CreateOn
FROM LMS_Loan_Master
WHERE
  Application_LoanPurpose = 'LAP' AND Application_CreateOn BETWEEN '2019-01-01' AND '2020-12-31'
ORDER BY
  Application_LoanAmount DESC;


In [66]:
# from llama_index.core.indices.struct_store.sql_query import (
#     SQLTableRetrieverQueryEngine,
# )
# from llama_index.core.objects import (
#     SQLTableNodeMapping,
#     ObjectIndex,
#     SQLTableSchema,
# )
# from llama_index.core import VectorStoreIndex

# # set Logging to DEBUG for more detailed outputs
# table_node_mapping = SQLTableNodeMapping(sql_database)
# table_schema_objs = [
#     (SQLTableSchema(table_name="Customer", table_node_mapping=table_node_mapping)),
# ]  # add a SQLTableSchema for each table

# obj_index = ObjectIndex.from_objects(
#     table_schema_objs,
#     table_node_mapping,
#     VectorStoreIndex,
# )
# query_engine = SQLTableRetrieverQueryEngine(
#     sql_database, obj_index.as_retriever(similarity_top_k=4, llm = Settings.llm), 
# )

In [70]:
# response = query_engine.query("show me loans which have loan amount greater than 500000")
response = query_engine.query("show me all male customers")

In [99]:
# print(response.response)

In [86]:
# print(str(response.metadata))

{'19a0f0e0-d231-41f2-a0fe-4b5f8df5a7bd': {}, 'sql_query': "SELECT DISTINCT `Customer_FirstName`, `Customer_LastName` FROM `Customer` WHERE `Customer_Gender` = 'M'"}


In [100]:
# response.metadata["sql_query"]

In [101]:
# sql_query = response.metadata["sql_query"]

# query Tester

In [2]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("""
    SELECT
  Loan_Id,
  LoanAcNo,
  LoanClosed_Date
FROM LMS_Loan_Master
WHERE
  LoanClosed_Date BETWEEN '2024-05-01' AND '2024-09-30';
    """))
    for row in rows:
        print(row)

In [ ]:
"""
SELECT
  Customer.Customer_FirstName,
  Customer.Customer_LastName,
  LMS_Loan_Master.Application_No,
  LMS_Loan_Master.Application_InterestAmt
FROM Customer
JOIN LMS_Loan_Master
  ON Customer.CustomerId = LMS_Loan_Master.Application_Account_CustomerId
WHERE
  LMS_Loan_Master.Application_Status = 'Closed';
  """

In [ ]:
SELECT
  Customer.Customer_FirstName,
  Customer.Customer_LastName,
  LMS_Loan_Master.Application_LoanAmount,
  LMS_Loan_Master.Application_InterestAmt,
  LMS_Loan_Master.LoanClosed_Date
FROM Customer
JOIN LMS_Loan_Master
  ON Customer.CustomerId = LMS_Loan_Master.Application_Account_CustomerId
WHERE
  LMS_Loan_Master.LoanClosed_Date IS NOT NULL
ORDER BY
  LMS_Loan_Master.LoanClosed_Date DESC;


In [ ]:
SELECT
  c.Customer_FirstName,
  c.Customer_LastName,
  l.Application_LoanAmount,
  l.Application_InterestAmt,
  l.LoanClosed_Date
FROM Customer AS c
JOIN LMS_Loan_Master AS l
  ON c.CustomerId = l.Application_Account_CustomerId
WHERE
  l.LoanClosed_Date BETWEEN '2022-06-01' AND '2023-04-30';

In [ ]:
SELECT
  CaseNo,
  Amount
FROM Acc_Voucher_Details
WHERE
  Amount = 1500 AND TranType = 'DR' AND CaseNo BETWEEN '4595' AND '4486';

In [1]:
import os
from dataclasses import dataclass

@dataclass
class SqlConnetionConfig:
    CONNECTION_URI: str


In [7]:
from sqlalchemy import create_engine
class SqlConnection:
    def __init__(self):
        pass
    
    def establish_connection(self):
        connection_config = SqlConnetionConfig(
            CONNECTION_URI = f"""mssql+pymssql://{os.getenv("USERNAME")}:{os.getenv("ENCODED_PASSWORD")}@{os.getenv("IP")}/{os.getenv("DATABASE_NAME")}"""
            )
        
        engine = create_engine(connection_config.CONNECTION_URI, pool_size=5, max_overflow=10)
        return engine

In [8]:
sql_conn = SqlConnection()

In [9]:
engine = sql_conn.establish_connection()

In [33]:
from sqlalchemy import text
import sys, logging
try:
    with engine.connect() as con:
        rows = con.execute(text("""
        SELECT lakfnek
        """))
        for row in rows:
            if sys.getsizeof(row) != 0:
                logging.info("Connection established")
except:
    logging.error("Connection failed")

ERROR:root:Connection failed


In [34]:
if engine:
    print("Connection established")

Connection established
